# Divide and Conquer method
https://arxiv.org/pdf/2205.11762

The first step is to divide a graph into a number of subgraphs with no more vertices than the number of available qubits on our quantum computer.
To do this we use the code for the Graph class given in the paper (utilities.py).

In [1]:
from utilities import *
from QAOA import *
from QAOA_square import *

### Create a graph

In [9]:
graph = [(3, 4), (3, 1), (3, 2), (4, 14), (4, 11), (8, 15), (8, 2), (8, 11), (15, 5), (15, 9), (2, 5), (5, 6), (1, 12), (1, 10), (12, 7), (12, 0), (13, 14), (13, 7), (13, 9), (14, 6), (6, 11), (7, 10), (10, 0), (9, 0)]
G = Graph(v=list(range(16)), edges=graph)

## Run normal QAOA to compare results

In [4]:
value_qaoa, sol_qaoa = qaoa(G, layer_count=2)
print(value_qaoa)
print(sol_qaoa)

2.5
001110


## Run with divide and conquer

The following functions are used to take the output from <code>quaoa_square</code> and recontsruct the final partitioning.

In [5]:
def flatten_2d_list(l):
    flattened = []
    for i in l:
        for j in i:
            flattened.append(j)
    return flattened

def contract_level(sols, sort = True):
    """
    Take a dictionary from a divide and conquer solution and contract the last level

    Parameters:
        sols: Dict{int: List[String]}
    """
    level = max(sols.keys())
    merged_sol = []
    for i, s in enumerate(sols[level - 1]['sol']):
        # find index corresponding to this subgraph in the next level
        idx = 0
        for j, v in enumerate(sols[level]['v']):
            if v == i:
                idx = j
                break
        
        for b in s:     
            if sols[level]['sol'][idx] == '0':
                merged_sol.append(b)
            else:
                merged_sol.append('0' if b == '1' else '1')

    merged_sol = ''.join(merged_sol)
    nodes = flatten_2d_list(sols[level - 1]['v'])
    merged_sol_sorted = [-1 for _ in range(len(merged_sol))]
    for n, p in zip(nodes, merged_sol):
        merged_sol_sorted[n] = p
    merged_sol_sorted = ''.join(merged_sol_sorted)
    sol_dict = {i: sols[i] for i in range(level - 1)}
    if sort:
        sol_dict[level - 1] = {'sol': merged_sol_sorted, 'v': sorted(list(nodes))}
    else:
        sol_dict[level - 1] = {'sol': merged_sol, 'v': list(nodes)}
    return sol_dict

def contract_solution(sols, sort = True):
    while len(sols) > 1:
        sols = contract_level(sols, sort)
    return sols


In [26]:
def get_cost(x_vec, graph):
    cost = 0
    for edge in graph:
        n1, n2 = edge
        cost += int(x_vec[n1])*(1-int(x_vec[n2])) + int(x_vec[n2])*(1-int(x_vec[n1]))

    return cost

In [29]:
value, sols = qaoa_square(G, depth=2, sub_size=8)

print(value)
print(sols)
final_partition = contract_solution(sols, sort=True)[0]['sol']
cost = get_cost(final_partition_vec, graph)

print("Final solution:", final_partition)
print("Cost:", cost)

18.5
{0: {'sol': ['00001111', '00101011'], 'v': [[8, 4, 6, 7, 3, 11, 12, 5], [1, 15, 9, 13, 10, 0, 2, 14]]}, 1: {'sol': '01', 'v': [0, 1]}}
Final solution: 1101010000011101
Cost: 19
